En el siguiente notebook vamos a trabajar con PySpark. En él, vamos a usar las transformaciones y las acciones para analizar un pequeño conjunto de datos.

El primer paso que debemos dar es conectar Google Colab con nuestro Google Drive. Para ello, lanzaremos el siguiente trozo de código.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


Una vez hemos conectado Google Drive, pasamos a instalar Apache Spark en nuestro notebook. En concreto, vamos a trabajar con la versión 3.5.0.

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q http://apache.mirrors.pair.com/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

In [ ]:
!ls


drive  sample_data  spark-3.5.0-bin-hadoop3.tgz


In [ ]:
!tar xf spark-3.5.0-bin-hadoop3.tgz

Además de la instalación habitual, hay un paso más que debemos dar, y es la instalación de la librería *findspark*. Esta librería nos permitirá encontrar la instalación de Apache Spark en nuestro sistema.

Establecemos también un par de variables de entorno.

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

In [ ]:
import findspark
findspark.init()

In [ ]:
findspark.find()

'/content/spark-3.5.0-bin-hadoop3'

Una vez tenemos instalado nuestro framework, es momento de crear nuestra variable SparkSession, de la cual parten la mayoría de funcionalidades de SparkSQL

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Pyspark_SQL")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
spark

### Lectura de datos

Ahora sí, podemos comenzar a trabajar con SparkSQL. Comenzaremos leyendo nuestro conjunto de datos, el cual contiene información de valoraciones de restaurantes.

In [ ]:
!pwd

/content


In [ ]:
import pandas as pd

data_path = '/content/drive/MyDrive/TokioSchool/'

data = spark.read.options(inferSchema='True',delimiter=',', header=True).csv (data_path + 'CSV_stocks_2021.csv')


In [ ]:
data

DataFrame[ticker: string, open: double, high: double, low: double, close: double, volume: int, dividends: double, stock splits: int, date: date, ccy: string]

### Análisis estadísico

In [ ]:
data.printSchema()

root
 |-- ticker: string (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: integer (nullable = true)
 |-- dividends: double (nullable = true)
 |-- stock splits: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- ccy: string (nullable = true)



Para usar solo algunas columnas podemos usar el método *select*

Obtención de datos

*   a) Realizaremos un filtro los datos de que se correspondan con el valor ‘GOLD’ en la columna ‘ticker’
*   b) Calcularemos el valor medio de la columna ‘open’
*   c) Calcularemos el valor medio de la columna ‘low’




In [ ]:
# a) Realizaremos un filtro los datos de que se correspondan con el valor ‘GOLD’ en la columna ‘ticker’
data.filter(data.ticker == 'GOLD').show()

+------+------------------+------------------+------------------+------------------+--------+---------+------------+----------+---+
|ticker|              open|              high|               low|             close|  volume|dividends|stock splits|      date|ccy|
+------+------------------+------------------+------------------+------------------+--------+---------+------------+----------+---+
|  GOLD|22.396357876868176|22.415624002019847| 21.86655229642893|21.943614959716797|18088000|      0.0|           0|2020-12-31|USD|
|  GOLD| 22.97432757969563|  23.8894464291543| 22.75277266388846| 23.71605682373047|41044700|      0.0|           0|2021-01-04|USD|
|  GOLD|24.014676568290795| 24.03394269418461|23.205517653939516| 23.38854217529297|30884800|      0.0|           0|2021-01-05|USD|
|  GOLD|23.186251350241353|23.667893469979873|  23.0224929560375|  23.6004638671875|30367800|      0.0|           0|2021-01-06|USD|
|  GOLD| 23.63899500309041|23.947245651204845| 23.35964170491246|23.86054992

In [ ]:
# b)Calcularemos el valor medio de la columna ‘open’
data.agg({'open': 'mean'}).show()


+-----------------+
|        avg(open)|
+-----------------+
|278.0612842540383|
+-----------------+



In [78]:
# c)Calcularemos el valor medio de la columna ‘open’
data.agg({'low': 'mean'}).show()

+------------------+
|          avg(low)|
+------------------+
|274.65344625831915|
+------------------+

